In [13]:
from typing import *

In [2]:

class RomanToDecimal:
    letters: Dict[str, int] = dict(i=1, v=5, x=10, l=50, c=100, d=500, m=1000)
    total: int
    roman_number: str

    def __init__(self, roman_number: str) -> None:
        self.roman_number = roman_number.lower()
        self.number_list = [self.letters[letter] for letter in self.roman_number]
        self.resolve()

    def __repr__(self) -> str:
        return f'{self.roman_number.upper()} = {self.suma}'

    def __getitem__(self, index) -> Optional[int]:
        try:
            return self.number_list[index]
        except IndexError:
            return None

    def __iter__(self) -> Iterator[Tuple[int, Optional[int]]]:
        for i, curr in enumerate(self.number_list):
            yield curr, self[i+1] 

    def resolve(self):
        self.suma = sum([curr if next is None or curr >= next else -curr for curr, next in self])

In [3]:
RomanToDecimal('XXIV')

XXIV = 24

In [4]:
class DecimalToRoman:
    number: int
    roman_number: str = ''
    numbers: Dict[int, str] = {
        1: 'i', 4: 'iv', 5: 'v', 9: 'ix', 10: 'x', 40: 'xl',
        50: 'l', 90: 'xc', 100: 'c', 400: 'cd', 500: 'd', 900: 'cm', 1000: 'm'
    }

    def __init__(self, number: int) -> None:
        self.number = number
        self.resolve()
        
    def __repr__(self) -> str:
        return f'{self.number} = {self.roman_number}'

    def __iter__(self) -> Iterator[Tuple[int, str]]:
        for k, v in reversed(self.numbers.items()):
            yield k, v

    def resolve(self):
        total = self.number
        self.roman_number = ''
        while total > 0:
            for number, letter in self:
                if total / number >= 1:
                    total -= number
                    self.roman_number += letter.upper()
                    break

In [5]:
DecimalToRoman(49)

49 = XLIX

In [96]:
from typing import *


def _is_roman(string: str, letters: List[str]) -> bool:
    import re
    izi_regex = re.compile(r'^(?=[MDCLXVI])M*(C[MD]|D?C{0,3})(X[CL]|L?X{0,3})(I[XV]|V?I{0,3})$')
    from itertools import groupby
    groups = groupby(string)
    # All letters are romman letters
    if not all([letter in letters for letter in string]):
        return False
    # If any of the romman letters is repeated more than 3 times in a row, is not roman number
    if any([count > 3 for _, count in [(label, sum(1 for _ in group)) for label, group in groups]]):
        return False
    # If any of the two letter romman numbers is repeated more than once, is not roman number
    if any([len(string.split(double_letter)) - 1 > 1 for double_letter in filter(lambda x: len(x) == 2, letters)]):
        return False
    if not izi_regex.match(string.upper()):
        return False
    return True

class NotRomanNumberException(Exception):
    pass

class RomanNumber:
    def __init__(self, roman_number: str) -> None:
        self.roman_number = roman_number.lower()

    def __str__(self) -> str:
        return self.roman_number.upper()

    def __len__(self) -> int:
        return len(self.roman_number)
    
    def __iter__(self) -> Iterator[Tuple[Optional[str], str, Optional[str]]]:
        for i, curr in enumerate(self.roman_number):
            yield self[i - 1], curr, self[i + 1]
        
    def __getitem__(self, index) -> Optional[str]:
        if index < 0: return None
        try:
            return self.roman_number[index]
        except IndexError:
            return None

class RomanToDecimal2:
    letters: Dict[str, int] = dict(i=1, iv=4, v=5, ix=9, x=10, xl=40, l=50, xc=90, c=100, cd=400, d=500, cm=900, m=1000)

    def __init__(self, roman_number: str) -> None:
        if not isinstance(roman_number, str):
            raise NotRomanNumberException(f'{roman_number=} must be a "str" instance')
        if not _is_roman(roman_number.lower(), self.letters.keys()):
            raise NotRomanNumberException(f'{roman_number=} is not a valid roman number')
        self.roman_number = RomanNumber(roman_number)

    def __repr__(self) -> str:
        return str(self)

    def __str__(self) -> str:
        if not hasattr(self, 'result'): self()
        return f'{self.roman_number} = {self.result}'

    def __len__(self) -> int:
        return len(self.roman_number)

    def __getitem__(self, index) -> int:
        return self.letters[index]
        
    def __contains__(self, key) -> bool:
        return key in self.letters

    def __call__(self) -> int:
        string = ''
        for previous_letter, letter, next_letter in self.roman_number:
            if letter and next_letter and letter + next_letter in self:
                continue
            if previous_letter and letter and (rsl := previous_letter + letter) in self:
                string += 'i' * self[rsl]
                continue
            string += 'i' * self[letter]
        self.result = len(string)
        return self.result

a = RomanToDecimal2(1)
a

NotRomanNumberException: roman_number=1 must be a "str" instance

'V'